In [0]:
%sql

SELECT * FROM flights_project.silver.silver_passengers

## Fetching Parameters

In [0]:
#Our Key Column Key
import ast

key_column = "['passenger_id']"
key_column_list = ast.literal_eval(key_column)

#Our CDC Column
cdc_column = 'update_date'



#Back-Dated Refresh
backdated_refresh = ""

#Source Object
source_object = 'silver_passengers'

#Source Sxhema
source_schema = 'silver'

target_object = 'DimPassengers'

target_schema = 'gold'

surrogate_key = 'DimPassengersKey'

## Data Ingestion 

## Last Load Data

In [0]:
if len(backdated_refresh) == 0:
  
  if spark.catalog.tableExists(f"flights_project.{target_schema}.{target_object}"):
    last_load = spark.sql(f"SELECT max({cdc_column}) FROM flights_project.{target_schema}.{target_object}").collect()[0][0]

  else:
    last_load = "1800-01-01 00:00:00"

else:
   last_load = backdated_refresh
   #Run last_load


last_load

In [0]:
df_src = spark.sql(f"SELECT * FROM flights_project.{target_schema}.{target_object} WHERE {cdc_column} > '{last_load}'")

df_src.display()


## Old VS New Records

In [0]:
if spark.catalog.tableExists(f"flights_project.{target_schema}.{target_object}"):
    key_column_string_incr = ', '.join(key_column_list)
    df_trg = spark.sql(f"SELECT * FROM flights_project.{target_schema}.{target_object}")

else:
    key_column_string_init = [f"'' AS {i}" for i in key_column_list]
    key_column_string_init = ', '.join(key_column_string_init)
    df_trg = spark.sql(f"""
        SELECT {key_column_string_init}, CAST('0' AS INT) AS {surrogate_key}, CAST('1800-01-01 00:00:00' AS TIMESTAMP) AS scd_create_date, CAST('1800-01-01 00:00:00' AS TIMESTAMP) AS scd_update_date
        FROM flights_project.{source_schema}.{source_object} WHERE 1=0
    """)

In [0]:
df_trg.display()

In [0]:
key_column_string_incr = ', '.join(key_column_list)

## Join Condition

In [0]:
join_cond = 'AND '.join([f"src.{i} = trg.{i}" for i in key_column_list])

In [0]:
df_src.createOrReplaceTempView('src')
df_trg.createOrReplaceTempView('trg')

df_join = spark.sql(f"""
          SELECT src.*, trg.scd_create_date, trg.scd_update_date, trg.{surrogate_key}
          FROM src
          LEFT JOIN trg
          ON {join_cond} """)

In [0]:
df_join.display()

In [0]:
from pyspark.sql.functions import *


In [0]:
#Old Records
df_old = df_join.filter(col(surrogate_key).isNotNull())

#New Records
df_new = df_join.filter(col(surrogate_key).isNull())

## Preparing DF_OLD

In [0]:
df_old_cur = df_old.withColumn("scd_update_date", current_timestamp())\
    .withColumn("scd_create_date", col("scd_update_date").cast("timestamp"))

In [0]:
df_old_cur.display()

## Preparing DF_New

In [0]:
df_new.display()

In [0]:
if spark.catalog.tableExists(f"flights_project.{target_schema}.{target_object}"):
  spark.sql(f"""
          SELECT MAX({surrogate_key})
          FROM flights_project.{target_schema}.{target_object}""").collect()[0][0]
  df_new_cur = df_new.withColumn(f"{surrogate_key}", lit(max_surrogate_key)+ lit(1)+  monotonically_increasing_id())\
      .withColumn("scd_create_date", current_timestamp())\
      .withColumn("scd_update_date", current_timestamp())
          
else:
  max_surrogate_key = 0
  df_new_cur = df_new.withColumn(f"{surrogate_key}", lit(max_surrogate_key)+ lit(1)+  monotonically_increasing_id())\
      .withColumn("scd_create_date", current_timestamp())\
      .withColumn("scd_update_date", current_timestamp())

In [0]:
df_old_cur.display()

In [0]:
df_new_cur.display()


In [0]:
df_union = df_old_cur.unionByName(df_new_cur)

In [0]:
df_union.display()

## Upsert

In [0]:
from delta.tables import DeltaTable

In [0]:
if spark.catalog.tableExists(f"flights_project.{target_schema}.{target_object}"):
  dlt_obj = DeltaTable.forName(spark, f"flights_project.{target_schema}.{target_object}")
  dlt_obj.alias("trg").merge(df_union.alias("src"), f"trg.{surrogate_key} = src.{surrogate_key}")\
      .whenMatchedUpdateAll(condition= f"src.{cdc_column} >= trg.{cdc_column}")\
      .whenNotMatchedInsertAll()\
      .execute()
else:
  df_union.write.format("delta").mode("overwrite").option("path", f"abfss://gold@dbdatalakejess.dfs.core.windows.net/{target_object}").saveAsTable(f"flights_project.{target_schema}.{target_object}")


In [0]:
df = spark.sql(f"""
SELECT *  FROM flights_project.{target_schema}.{target_object}
""")

In [0]:
df.display()